In [ ]:
import ee
import geemap
import pandas as pd
import os
import rasterio
import numpy as np
from PIL import Image
from datetime import datetime, timedelta

# === Initialize Earth Engine ===
ee.Initialize(project='ee-dharanideivasihamani')

# === Config ===
INPUT_CSV = rf"datasets\vibe_full_features.csv"
OUTPUT_CSV = rf"datasets\vibe_full_features_with_sentinel.csv"
SAVE_FOLDER = rf"datasets\sentinel_tiles"
TILE_SIZE_METERS = 2240  # 224px * 10m resolution

# === Create folders for each class ===
df = pd.read_csv(INPUT_CSV)
classes = df["vibe_class"].unique()
for cls in classes:
    os.makedirs(os.path.join(SAVE_FOLDER, cls), exist_ok=True)

# === Define date range for composite ===
end_date = datetime.today()
start_date = end_date - timedelta(days=90)
start = start_date.strftime('%Y-%m-%d')
end = end_date.strftime('%Y-%m-%d')

# === Sentinel-2 RGB composite ===
def get_sentinel_image(lat, lon):
    point = ee.Geometry.Point([lon, lat])
    region = point.buffer(TILE_SIZE_METERS / 2).bounds()

    s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
          .filterBounds(region)
          .filterDate(start, end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
          .select(['B4', 'B3', 'B2']))

    composite = s2.mean().clip(region)

    if composite.bandNames().size().getInfo() == 0:
        return None, None

    return composite, region

# === Convert TIFF to real PNG ===
def convert_tif_to_png(tif_path, png_path):
    try:
        with rasterio.open(tif_path) as src:
            img = src.read([1, 2, 3])  # RGB bands
            img = np.transpose(img, (1, 2, 0))  # CHW → HWC
            img = np.clip(img, 0, 255).astype(np.uint8)
            im = Image.fromarray(img)
            im.save(png_path)
        return png_path
    except Exception as e:
        print(f"[ERROR] TIFF to PNG failed: {e}")
        return "NA"

# === Export both GeoTIFF and PNG ===
def export_sentinel_images(composite, region, out_path_base, out_path_tiff):
    try:
        viz_tif_path = out_path_base + "_viz.tif"           # temp visualized TIFF
        png_path = out_path_base + "_sentinel.png"
        tiff_path = out_path_tiff + "_sentinel.tif"         # raw GeoTIFF

        # Export visualized RGB as GeoTIFF
        rgb_viz = composite.visualize(min=0, max=3000, bands=["B4", "B3", "B2"])
        geemap.download_ee_image(
            rgb_viz, region=region, filename=viz_tif_path, scale=10, crs="EPSG:4326"
        )

        # Convert to real PNG
        png_path = convert_tif_to_png(viz_tif_path, png_path)

        
        # Export raw GeoTIFF (multiband)
        geemap.download_ee_image(
            composite, region=region, filename=tiff_path, scale=10, crs="EPSG:4326"
        )

        return png_path, tiff_path

    except Exception as e:
        print(f"[ERROR] Download failed for {out_path_base}: {e}")
        return "NA", "NA"

# === Process all locations ===
png_paths = []
tiff_paths = []

for idx, row in df.iterrows():
    lat, lon, vibe = row["lat"], row["lon"], row["vibe_class"]
    out_base_png = os.path.join(SAVE_FOLDER, vibe, f"{vibe}_{idx}")
    out_base_tiff = os.path.join(rf"D:\vibe_place_mapper\datasets\sentinel_tifs", vibe, f"{vibe}_{idx}")

    composite, region = get_sentinel_image(lat, lon)
    if composite is None:
        print(f"[SKIPPED] No image at index {idx} ({lat}, {lon})")
        png_paths.append("NA")
        tiff_paths.append("NA")
        continue

    png_path, tiff_path = export_sentinel_images(composite, region, out_base_png, out_base_tiff)
    png_paths.append(png_path)
    tiff_paths.append(tiff_path)

# === Update CSV ===
df["sentinel_png_path"] = png_paths
df["sentinel_tiff_path"] = tiff_paths
df.to_csv(OUTPUT_CSV, index=False)

print(f"Final dataset with Sentinel imagery saved to: {OUTPUT_CSV}")
